<a href="https://colab.research.google.com/github/klaxonlet/CE888_2020/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv')
print(df)

        74  -7.82   8.79  -9.66  -8.16  ...  99.22  -5.63  99.23  99.24  99.25
0      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
1       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
2       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
4      100  -6.17  -3.54   0.44  -8.50  ...  -3.35   0.05  -9.08  -5.05  -3.45
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24978   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24979   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24980   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 101 columns]


In [11]:
df = df.drop(df.columns[0],1)
print(df)

       -7.82   8.79  -9.66  -8.16  -7.52  ...  99.22  -5.63  99.23  99.24  99.25
0       4.08  -0.29   6.36   4.37  -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
1      99.00  99.00  99.00  99.00   9.03  ...  99.00  99.00  99.00  99.00  99.00
2      99.00   8.35  99.00  99.00   1.80  ...  99.00  99.00  99.00  99.00  99.00
3       8.50   4.61  -4.17  -5.39   1.36  ...   1.55   3.11   6.55   1.80   1.60
4      -6.17  -3.54   0.44  -8.50  -7.09  ...  -3.35   0.05  -9.08  -5.05  -3.45
...      ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977   0.44   7.43   9.08   2.33   3.20  ...   9.03   6.55   8.69   8.79   7.43
24978   9.13  -8.16   8.59   9.08   0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24979  99.00  99.00  99.00  99.00  -7.77  ...  99.00  99.00  99.00  99.00  99.00
24980  99.00  99.00  99.00  99.00  -9.71  ...  99.00  99.00  99.00  99.00  99.00
24981   2.43   2.67  -3.98   4.27  -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 100 columns]


In [12]:
arr = df.values
print(arr.shape)

(24982, 100)


In [13]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810381 (1810381,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [20]:
arr[idx[0][0], idx[1][0]]

2.14

In [21]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 15

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10) == 0:
        print(mse)

In [24]:
sgd()

25.846872188255023
24.933807631374037
24.881389402469008
24.832866079915732
24.78541970342406
24.736425044165095
24.682886237265464
24.620934829749597
24.54521456439903
24.44811693820425
24.31889425822264
24.142838391747937
23.901046953056948
23.571837961138307
23.135386000107882
22.582736977650605
21.92747252621278
21.21285526012288
20.50464350319032
19.868245276272653
19.344524920913013
18.941563206591375
18.643963179358035
18.42758421864574
18.2695988460155
18.152227965640254
18.062795445059372
17.992574974018048
17.935599938234507
17.88775997223773
17.84618232612245
17.808822648999257
17.774196096404552
17.74119996052515
17.708995910103514
17.67693137638388
17.644486946269723
17.6112412550912
17.57684780695149
17.54102001838211
17.50352196890779
17.46416309279609
17.422795515521262
17.379313029809204
17.333650891216344
17.285785743636882
17.2357350986127
17.18355591498415
17.12934197200597
17.07321990217319


In [25]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 3.647e+00,  2.460e+00,  3.508e+00, ...,  2.118e+00,  2.850e+00,
         3.016e+00],
       [ 7.191e+00,  7.432e+00,  7.960e+00, ...,  7.088e+00,  6.117e+00,
         8.095e+00],
       [ 2.045e+00,  3.566e+00,  2.527e+00, ...,  2.895e+00,  3.281e-01,
         3.811e+00],
       ...,
       [-7.724e-01, -1.445e+00, -1.778e+00, ..., -1.568e+00, -4.128e+00,
        -7.347e-01],
       [ 2.519e-01, -2.306e-01, -8.374e-01, ..., -3.806e-01, -1.975e+00,
         1.881e-03],
       [ 3.092e+00,  4.436e+00,  3.174e+00, ...,  3.793e+00,  8.213e-01,
         5.009e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns



In [29]:
comparison_data

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,-8.4.2,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,"(4.08, 3.6467346350299703)","(-0.29, 2.4604985048745354)","(6.36, 3.5082092038170685)","(4.37, 1.7100776974278329)","(-2.38, 2.193477013589042)","(-9.66, 3.5485290128242597)","(-0.73, 1.9982468043631865)","(-5.34, 1.7878422620449212)","(8.88, 3.5110583488059524)","(9.22, 3.8956843363477667)","(6.75, 3.891746991825015)","(8.64, 3.4212160015790247)","(4.42, 0.2860401952202587)","(7.43, 3.1730533779256986)","(4.56, 0.7194145216176264)","(-0.97, 0.12809239727564312)","(4.66, 0.27177582492758373)","(-0.68, 1.9461005543490564)","(3.3, 2.5751230225978605)","(-1.21, 1.9720509484389765)","(0.87, 2.8382327942641528)","(8.64, 3.257597948816048)","(8.35, 3.2790243557657184)","(9.17, 2.440517768342028)","(0.05, 2.7799169834057293)","(7.57, 2.8656631223934443)","(4.71, 3.4380821034216664)","(0.87, 4.279072906935527)","(-0.39, 3.2762536453647364)","(6.99, 3.2146064372881566)","(6.5, 2.933173805096849)","(-0.92, 3.887312872516908)","(7.14, 2.632183635071766)","(9.03, 2.8613483891089655)","(-1.8, 3.889519614060795)","(0.73, 3.3451812444778533)","(7.09, 2.6018523167569176)","(3.4, 3.917852638071054)","(-0.87, 3.0777024147011818)","(7.91, 3.6414072564157585)",...,"(-6.7, 3.9287459341863404)","(-3.35, 4.221453203896514)","(-9.03, 3.951751575344851)","(4.47, 2.999655122034569)","(4.08, 3.295469659190592)","(-3.83, 3.7208757500277603)","(8.74, 3.245267129679754)","(1.12, 4.231519893126973)","(0.78, 3.4611158550485848)","(7.52, 3.194447456692907)","(-5.0, 2.6273251103672903)","(2.77, 3.484801606195283)","(8.3, 2.799736811141887)","(7.77, 2.255699031991668)","(7.33, 2.196613947306642)","(6.21, 3.0180214077807324)","(7.72, 2.4617472687474877)","(8.98, 3.596423730068647)","(8.64, 2.671735037770724)","(8.2, 2.5874534228376245)","(3.93, 2.970671666362732)","(4.85, 3.999739506015492)","(4.85, 3.2585446326535226)","(6.07, 3.8012591822819934)","(8.98, 3.190232841640875)","(4.51, 3.565853365737917)","(-0.05, 3.0361915863450037)","(3.69, 4.047173667637636)","(4.56, 4.305885682510408)","(0.58, 3.9109440448704795)","(2.82, 4.237373503366532)","(-4.95, 3.513045441350053)","(-0.29, 3.950549836461828)","(7.86, 4.152640020907962)","(-0.19, 3.204218499996258)","(-2.14, 3.657274437653546)","(3.06, 3.862973668885005)","(0.34, 2.1178228849231546)","(-4.32, 2.8504324682079947)","(1.07, 3.016409992065407)"
1,"(99.0, 7.19094945429273)","(99.0, 7.431783880016686)","(99.0, 7.959640909856757)","(99.0, 6.8202904946568275)","(9.03, 5.692322729762228)","(9.27, 8.383847215923657)","(9.03, 4.517481888531234)","(9.27, 2.893867659432144)","(99.0, 7.927162973402751)","(99.0, 7.550019125950788)","(7.33, 8.920654009476767)","(7.57, 8.878385126624032)","(9.37, 1.6379454427014504)","(6.17, 9.162233409975538)","(-6.36, 0.9771169743324675)","(-6.89, 0.6196541127601912)","(-7.86, 2.7734381017529035)","(9.03, 3.7975119488544773)","(9.03, 5.178414599424008)","(9.03, 4.97120490759309)","(7.28, 8.97438209673663)","(99.0, 8.1251585425777)","(8.25, 7.202593124684083)","(99.0, 6.337142892581982)","(99.0, 7.873773651971514)","(7.48, 8.346128222122468)","(7.28, 7.970877079306537)","(7.28, 8.670235890054256)","(8.93, 7.7471758586775294)","(99.0, 7.824068919946835)","(6.17, 8.370301464859564)","(7.28, 8.152524548296265)","(99.0, 7.21570704150014)","(99.0, 8.423141791228035)","(8.98, 8.505280638504525)","(7.33, 7.655372042211836)","(99.0, 5.530108913183588)","(6.17, 7.865281672081599)","(9.08, 8.339817106926162)","(7.33, 8.430767440138657)",...,"(6.46, 8.80124550984361)","(7.28, 8.533150019973442)","(99.0, 7.528935681616694)",